### Report to 11020PHYS401200-Computational-Physics

### Part B : Build Transfer Matrix By Tensor Method ( Numpy vs Cytnx)

* Author : Yen-Tung Lin
* ID : 109022802

### Reference
* Complexity and Criticality, Kim Christensen

### Model

* Classical Ising Model in 2 D : E = J $\sum_{<i,j>}S^{z}_{i}S^{z}_{j}$


### To Calculate

* Transfer Matrix
* Eigenvalues of Transfer Matrix
* Corrlation length $\xi$=$\frac{1}{\log(\lambda_{0}/\lambda_{1})}$

### Import

In [41]:
import os
import math
import random
import cytnx as cy
from scipy.optimize import curve_fit
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

In [42]:
## Basic parameter
cnames = {                        
'blueviolet':           '#8A2BE2',
'brown':                '#A52A2A',
'burlywood':            '#DEB887',
'cadetblue':            '#5F9EA0',
'chocolate':            '#D2691E',
'crimson':              '#DC143C',
'cyan':                 '#00FFFF',
'darkblue':             '#00008B',
'darkcyan':             '#008B8B',
'darkolivegreen':       '#556B2F',
'darkorange':           '#FF8C00',
'darksalmon':           '#E9967A',
'darkseagreen':         '#8FBC8F',
'darkslateblue':        '#483D8B',
'darkslategray':        '#2F4F4F',
'darkturquoise':        '#00CED1',
'darkviolet':           '#9400D3',
'deeppink':             '#FF1493',
'deepskyblue':          '#00BFFF',
'dimgray':              '#696969',
'dodgerblue':           '#1E90FF',
'firebrick':            '#B22222',
'floralwhite':          '#FFFAF0',
'forestgreen':          '#228B22',
'fuchsia':              '#FF00FF',
'gainsboro':            '#DCDCDC',
'ghostwhite':           '#F8F8FF',
'gold':                 '#FFD700',
'greenyellow':          '#ADFF2F',
'honeydew':             '#F0FFF0',
'hotpink':              '#FF69B4',
'indianred':            '#CD5C5C',
'indigo':               '#4B0082',
'ivory':                '#FFFFF0',
'khaki':                '#F0E68C',
'orangered':            '#FF4500',
'orchid':               '#DA70D6'}
carr = []
for cmap in cnames.keys():                          ## color array
    carr.append(cmap)
marr = ['o-', 'x-', '^-', 's-', 'p-', '*-', 'h-']   ## marker array

## Construct transfer matrix through Tensor Methods

### Numpy Method

### 設定 M

In [43]:
t = 1
J = 1
beta = 1/t
M = np.array([[np.sqrt(math.cosh(beta*J)), np.sqrt(math.sinh(beta*J))],
                [np.sqrt(math.cosh(beta*J)),-np.sqrt(math.sinh(beta*J))]])

### 用 numpy 把 M 接成 T


#### $T_{np}$

In [44]:
T = np.zeros((2,2,2,2))

for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                T[i,j,k,l] = np.sum(M[:,i]*M[:,j]*M[:,k]*M[:,l])
print("T_shape",np.shape(T))
print("T",T)

# M[:,0]*M[:,1]逐項相乘

T_shape (2, 2, 2, 2)
T [[[[4.76219569 0.        ]
   [0.         3.62686041]]

  [[0.         3.62686041]
   [3.62686041 0.        ]]]


 [[[0.         3.62686041]
   [3.62686041 0.        ]]

  [[3.62686041 0.        ]
   [0.         2.76219569]]]]


#### $T+T=T_{np,y=2}$

In [45]:
np_TT = np.tensordot(T,T,(1,3))
np_TT = np.transpose(np_TT,(0,3,4,1,5,2))
np_TT = np_TT.reshape((4,2,4,2))
np_TT_trace = np.copy(np.trace(np_TT,axis1 = 1,axis2 =  3))
print(np.shape(np_TT))
print("np_TT",np_TT)
np.linalg.eigh( np.trace(np_TT,axis1 = 1,axis2 =  3) )


(4, 2, 4, 2)
np_TT [[[[22.6785078   0.        ]
   [ 0.         13.15411642]
   [ 0.         17.27181901]
   [13.15411642  0.        ]]

  [[ 0.         13.15411642]
   [17.27181901  0.        ]
   [13.15411642  0.        ]
   [ 0.         13.15411642]]]


 [[[ 0.         13.15411642]
   [17.27181901  0.        ]
   [13.15411642  0.        ]
   [ 0.         13.15411642]]

  [[17.27181901  0.        ]
   [ 0.         10.01809819]
   [ 0.         13.15411642]
   [10.01809819  0.        ]]]


 [[[ 0.         17.27181901]
   [13.15411642  0.        ]
   [17.27181901  0.        ]
   [ 0.         10.01809819]]

  [[13.15411642  0.        ]
   [ 0.         13.15411642]
   [ 0.         10.01809819]
   [13.15411642  0.        ]]]


 [[[13.15411642  0.        ]
   [ 0.         13.15411642]
   [ 0.         10.01809819]
   [13.15411642  0.        ]]

  [[ 0.         13.15411642]
   [10.01809819  0.        ]
   [13.15411642  0.        ]
   [ 0.          7.62972504]]]]


(array([ 0.94512664,  0.98168436, 53.59815003, 55.67133903]),
 array([[-0.60208675, -0.        ,  0.        , -0.79843068],
        [ 0.        ,  0.70710678, -0.70710678,  0.        ],
        [ 0.        , -0.70710678, -0.70710678,  0.        ],
        [ 0.79843068,  0.        ,  0.        , -0.60208675]]))

## CYTNX Method

### Define function : Print Diagram and Element

In [46]:
def ut_print(ut):
    ut.print_diagram()
    print(ut.get_block().numpy())

### Dictionary to store Tensor

In [47]:
Uni_T_dic = {}
Uni_T_trace_dic = {}
EIGVLE_dic = {}

#### $T_{cy}$

In [48]:
t=1
m = np.array([[np.sqrt(np.cosh(1/t)),np.sqrt(np.sinh(1/t))],
                [np.sqrt(np.cosh(1/t)),-np.sqrt(np.sinh(1/t))]])
m_transpose = np.copy(m.transpose())
CyT_M = cy.from_numpy(m)
Uni_M = cy.UniTensor(CyT_M, rowrank = 1)
Uni_M.print_diagram()

CyT_M_transpose = cy.from_numpy(m_transpose)
Uni_M_transpose = cy.UniTensor(CyT_M_transpose, rowrank = 1)
Uni_M_transpose.print_diagram()

Uni_M_up = Uni_M_transpose.clone()
Uni_M_up.set_name("Uni_M_up")
ut_print(Uni_M_up)

Uni_M_left = Uni_M_transpose.clone()
Uni_M_left.set_name("Uni_M_left")
ut_print(Uni_M_left)

Uni_M_right = Uni_M.clone()
Uni_M_right.set_name("Uni_M_right")
ut_print(Uni_M_right)

Uni_M_down = Uni_M.clone()
Uni_M_down.set_name("Uni_M_down")
ut_print(Uni_M_down)

delta = np.zeros((2,2,2,2))
delta[0,0,0,0] = delta[1,1,1,1] = 1

Delta = cy.from_numpy(delta)
UniDelta = cy.UniTensor(Delta, rowrank = 2)
UniDelta.set_labels([0,1,2,3])
UniDelta.set_name("UniDelta")
ut_print(UniDelta)

M_network = cy.Network("network/T.net")
M_network.PutUniTensors(['mu','ml','mr','md','delta'], [Uni_M_up,Uni_M_left,Uni_M_right,Uni_M_down,UniDelta])
print(M_network)
Uni_T=M_network.Launch()
Uni_T.set_name("Uni_T")
CyT_T = Uni_T.get_block()
ut_print(Uni_T)

Uni_T_trace = Uni_T.clone().Trace(0, 3)

Uni_T_dic[(1,1)] = Uni_T
Uni_T_trace_dic[(1,1)] = Uni_T_trace
EIGVLE_dic[(1,1)], v = np.linalg.eigh(Uni_T_trace.get_block().numpy())


-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
-----------------------
tensor Name : Uni_M_up
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 1  
           \             /     
            -------------      
[[ 1.24220797  1.24220797]
 [ 1.08406697 -1.08406697]]
-----------------------
tensor Name : Uni_M_left
tensor Rank : 2
bl

### $T+T=T_{cy,y=2}$

In [49]:
Ymerge_network = cy.Network("network/Ymerge.net")
print(Ymerge_network)

TU =  Uni_T.clone()
TD =  Uni_T.clone()


Ymerge_network.PutUniTensors(['TU','TD'], [TU,TD])
Uni_Ty2=Ymerge_network.Launch()
Uni_Ty2.set_name("Uni_TT")
ut_print(Uni_Ty2)

Uni_Ty2.combineBonds([1,10])
Uni_Ty2.combineBonds([2,11])
ut_print(Uni_Ty2)

Uni_Ty2_trace = Uni_Ty2.clone().Trace(0, 3)
ut_print(Uni_Ty2_trace)
Uni_Ty2_trace.set_labels([0,1])
ut_print(Uni_Ty2_trace)

Uni_T_dic[(1,2)] = Uni_Ty2
Uni_T_trace_dic[(1,2)] = Uni_Ty2_trace
EIGVLE_dic[(1,2)], v = np.linalg.eigh(Uni_Ty2_trace.get_block().numpy())


==== Network ====
[x] TU : 0 1 ; 2 -1 
[x] TD : -1 10 ; 11 3 
TOUT : 0 1 10 ; 2 11 3 
ORDER : 

-----------------------
tensor Name : Uni_TT
tensor Rank : 6
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 11 
           |             |     
    10 ____| 2         2 |____ 3  
           \             /     
            -------------      
[[[[[[22.6785078   0.        ]
     [ 0.         17.27181901]]

    [[ 0.         13.15411642]
     [13.15411642  0.        ]]]


   [[[ 0.         17.27181901]
     [17.27181901  0.        ]]

    [[13.15411642  0.        ]
     [ 0.         10.01809819]]]]



  [[[[ 0.         13.15411642]
     [13.15411642  0.        ]]

    [[17.27181901  0.        ]
     [ 0.         13.15411642]]]


   [[[13.15411642  0.        ]
     [ 0.         10.01809819]]

    [[ 0.         

## 比較 numpy 跟 Cytnx 的結果

### Elements Comparison : $T_{np}$ vs $T_{cy}$

In [50]:
print("Numpy Method\n",T)
print("Cytnx Method\n",Uni_T.get_block().numpy())

Numpy Method
 [[[[4.76219569 0.        ]
   [0.         3.62686041]]

  [[0.         3.62686041]
   [3.62686041 0.        ]]]


 [[[0.         3.62686041]
   [3.62686041 0.        ]]

  [[3.62686041 0.        ]
   [0.         2.76219569]]]]
Cytnx Method
 [[[[4.76219569 0.        ]
   [0.         3.62686041]]

  [[0.         3.62686041]
   [3.62686041 0.        ]]]


 [[[0.         3.62686041]
   [3.62686041 0.        ]]

  [[3.62686041 0.        ]
   [0.         2.76219569]]]]


### Elements Comparison : $T_{np,Ytrace}$ vs $T_{cy,Ytrace}$

In [51]:
print(np.trace(T,axis1 = 1,axis2 =  3))

[[8.3890561 0.       ]
 [0.        6.3890561]]


In [52]:
ut_print(Uni_T_trace)

-----------------------
tensor Name : Uni_T
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     1 ____| 2         2 |____ 2  
           \             /     
            -------------      
[[8.3890561 0.       ]
 [0.        6.3890561]]


### Eigvalue Comparison : $T_{np,Ytrace}$ vs $T_{cy,Ytrace}$

In [53]:
np.linalg.eigh( np.trace(T,axis1 = 1,axis2 =  3) )

(array([6.3890561, 8.3890561]),
 array([[0., 1.],
        [1., 0.]]))

In [54]:
np.linalg.eigh(Uni_T_trace.get_block().numpy())

(array([6.3890561, 8.3890561]),
 array([[0., 1.],
        [1., 0.]]))

### Elements Comparison : $T_{np,y=2}$ vs $T_{cy,y=2}$

In [55]:
print("np_TT\n")
print(np_TT)

np_TT

[[[[22.6785078   0.        ]
   [ 0.         13.15411642]
   [ 0.         17.27181901]
   [13.15411642  0.        ]]

  [[ 0.         13.15411642]
   [17.27181901  0.        ]
   [13.15411642  0.        ]
   [ 0.         13.15411642]]]


 [[[ 0.         13.15411642]
   [17.27181901  0.        ]
   [13.15411642  0.        ]
   [ 0.         13.15411642]]

  [[17.27181901  0.        ]
   [ 0.         10.01809819]
   [ 0.         13.15411642]
   [10.01809819  0.        ]]]


 [[[ 0.         17.27181901]
   [13.15411642  0.        ]
   [17.27181901  0.        ]
   [ 0.         10.01809819]]

  [[13.15411642  0.        ]
   [ 0.         13.15411642]
   [ 0.         10.01809819]
   [13.15411642  0.        ]]]


 [[[13.15411642  0.        ]
   [ 0.         13.15411642]
   [ 0.         10.01809819]
   [13.15411642  0.        ]]

  [[ 0.         13.15411642]
   [10.01809819  0.        ]
   [13.15411642  0.        ]
   [ 0.          7.62972504]]]]


In [56]:
print("Uni_Ty2 get_block\n")
print(Uni_Ty2.get_block().numpy())

Uni_Ty2 get_block

[[[[22.6785078   0.        ]
   [ 0.         17.27181901]
   [ 0.         13.15411642]
   [13.15411642  0.        ]]

  [[ 0.         17.27181901]
   [17.27181901  0.        ]
   [13.15411642  0.        ]
   [ 0.         10.01809819]]

  [[ 0.         13.15411642]
   [13.15411642  0.        ]
   [17.27181901  0.        ]
   [ 0.         13.15411642]]

  [[13.15411642  0.        ]
   [ 0.         10.01809819]
   [ 0.         13.15411642]
   [13.15411642  0.        ]]]


 [[[ 0.         13.15411642]
   [13.15411642  0.        ]
   [17.27181901  0.        ]
   [ 0.         13.15411642]]

  [[13.15411642  0.        ]
   [ 0.         10.01809819]
   [ 0.         13.15411642]
   [13.15411642  0.        ]]

  [[17.27181901  0.        ]
   [ 0.         13.15411642]
   [ 0.         10.01809819]
   [10.01809819  0.        ]]

  [[ 0.         13.15411642]
   [13.15411642  0.        ]
   [10.01809819  0.        ]
   [ 0.          7.62972504]]]]


### Elements Comparison $T_{np,y=2,Ytrace}$ vs $T_{cy,y=2,Ytrace}$

In [57]:
print(np_TT_trace)

[[35.83262422  0.          0.         26.30823284]
 [ 0.         27.2899172  26.30823284  0.        ]
 [ 0.         26.30823284 27.2899172   0.        ]
 [26.30823284  0.          0.         20.78384145]]


In [58]:
ut_print(Uni_Ty2_trace)

-----------------------
tensor Name : Uni_TT
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 4         4 |____ 1  
           \             /     
            -------------      
[[35.83262422  0.          0.         26.30823284]
 [ 0.         27.2899172  26.30823284  0.        ]
 [ 0.         26.30823284 27.2899172   0.        ]
 [26.30823284  0.          0.         20.78384145]]


### Eigvalue Comparison $T_{np,y=2,Ytrace}$ vs $T_{cy,y=2,Ytrace}$

In [59]:
print("eigh check of np_TT\n")
np.linalg.eigh( np_TT_trace )

eigh check of np_TT



(array([ 0.94512664,  0.98168436, 53.59815003, 55.67133903]),
 array([[-0.60208675, -0.        ,  0.        , -0.79843068],
        [ 0.        ,  0.70710678, -0.70710678,  0.        ],
        [ 0.        , -0.70710678, -0.70710678,  0.        ],
        [ 0.79843068,  0.        ,  0.        , -0.60208675]]))

In [60]:
print("eigh check of Uni_TT_trace\n")
np.linalg.eigh(Uni_Ty2_trace.get_block().numpy())

eigh check of Uni_TT_trace



(array([ 0.94512664,  0.98168436, 53.59815003, 55.67133903]),
 array([[-0.60208675, -0.        ,  0.        , -0.79843068],
        [ 0.        ,  0.70710678, -0.70710678,  0.        ],
        [ 0.        , -0.70710678, -0.70710678,  0.        ],
        [ 0.79843068,  0.        ,  0.        , -0.60208675]]))

## Try to contract x direction ( cytnx only )

### $T_{y=2} + T_{y=2} = T_{x=2,y=2}$

In [61]:
Uni_TL = Uni_Ty2.clone()
Uni_TR = Uni_Ty2.clone()

Xmerge = cy.Network("network/Xmerge.net")
Xmerge.PutUniTensors(['TL','TR'], [Uni_TL,Uni_TR])

Uni_Tx2y2 = Xmerge.Launch()
Uni_Tx2y2.combineBonds([0,4])
Uni_Tx2y2.combineBonds([3,5])
ut_print(Uni_Tx2y2)
Uni_Tx2y2_trace = Uni_Tx2y2.clone().Trace(0,3)
EIGVLE_Tx2y2_trace, v = np.linalg.eigh(Uni_Tx2y2_trace.get_block().numpy())

Uni_T_dic[(2,2)] = Uni_Tx2y2
Uni_T_trace_dic[(2,2)] = EIGVLE_Tx2y2_trace
EIGVLE_dic[(2,2)] = EIGVLE_Tx2y2_trace



-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 4         4 |____ 2  
           |             |     
     1 ____| 4         4 |____ 3  
           \             /     
            -------------      
[[[[687.34549478   0.           0.         471.34651053]
   [  0.         523.47831195 471.34651053   0.        ]
   [  0.         471.34651053 454.39103592   0.        ]
   [471.34651053   0.           0.         346.06155748]]

  [[  0.         471.34651053 523.47831195   0.        ]
   [471.34651053   0.           0.         398.67802315]
   [454.39103592   0.           0.         358.97474785]
   [  0.         346.06155748 358.97474785   0.        ]]

  [[  0.         454.39103592 471.34651053   0.        ]
   [454.39103592   0.           0.         358.97474785]
   [471.34651053   0.           0.         346.06155748]
   [  0.    

### Compare eigenvalues of $T_{x=2,y=2}$ $T_{x=1,y=2}$ and $T_{x=1,y=1}$

In [62]:
EIGVLE_dic[(1,1)]

array([6.3890561, 8.3890561])

In [63]:
EIGVLE_dic[(1,2)]

array([ 0.94512664,  0.98168436, 53.59815003, 55.67133903])

In [64]:
print("EIGVLE_dic[(1,2)]**2",EIGVLE_dic[(1,2)]**2)
# EIGVLE_dic[(1,2)]**2

EIGVLE_dic[(1,2)]**2 [8.93264368e-01 9.63704185e-01 2.87276169e+03 3.09929799e+03]


In [65]:
print("EIGVLE_dic[(2,2)]",EIGVLE_dic[(2,2)])
# EIGVLE_dic[(2,2)]

EIGVLE_dic[(2,2)] [8.93264368e-01 9.63704185e-01 2.87276169e+03 3.09929799e+03]
